In [5]:
#importo librerías

import pandas as pd
from fastapi import FastAPI
import ast


In [6]:
#cardo los datasets

credits = pd.read_csv("Dataset/credits.csv")

movies_dataset = pd.read_excel("Dataset/movies_dataset.xlsx")

In [7]:
# TRANSFORMACIONES - Primer Transformación, desanidar columnas


def convert(obj):
    if isinstance(obj, str) and '{' in obj: 
        L=[]
        for i in ast.literal_eval(obj): 
            L.append(i['name']); 
        return L



def convert2(obj):
    if isinstance(obj, str) and '{' in obj:
        dic = ast.literal_eval(obj)
        return dic['name']



In [8]:
movies_dataset['genres'] = movies_dataset['genres'].apply(convert)
movies_dataset['belongs_to_collection'] = movies_dataset['belongs_to_collection'].apply(convert2)
movies_dataset['production_companies'] = movies_dataset['production_companies'].apply(convert)
movies_dataset['production_countries'] = movies_dataset['production_countries'].apply(convert)
movies_dataset['spoken_languages'] = movies_dataset['spoken_languages'].apply(convert)

In [9]:
# Siguientes transformaciones, completar los faltantes son 0 en las columnas revenue y budget

movies_dataset["revenue"] = movies_dataset["revenue"].fillna(0)
movies_dataset["budget"] = movies_dataset["budget"].fillna(0)


In [10]:
# Eliminar valores nulos en la columna "release_date"
movies_dataset.dropna(subset=["release_date"], inplace=True)

# Convertir la columna de fecha al formato de fecha de Pandas
movies_dataset['release_date'] = pd.to_datetime(movies_dataset['release_date'], errors='coerce')

# Crear la columna "release_year" con el año de la fecha
movies_dataset['release_year'] = movies_dataset['release_date'].dt.year



In [11]:
#Crear la columna con el retorno de inversión, llamada return con los campos revenue y budget, dividiendo estas dos últimas revenue / budget,
#cuando no hay datos disponibles para calcularlo, deberá tomar el valor 0.

# Definir la función para calcular el retorno de inversión
def calculate_return(row):
    
    if row['budget'] == 0:
        return 0
    
    elif row["revenue"] == 0:
        return 0
    
    else:
        return row['revenue'] / row['budget']
    


movies_dataset['return'] = movies_dataset.apply(calculate_return, axis=1)



In [12]:

# Elimino columnas no utilizadas
movies_dataset = movies_dataset.drop(columns = ["video","imdb_id","adult","original_title","poster_path","homepage"])

In [13]:
# hago un duplicado del df para trabajar mejor
credits2 = credits

credits2 = credits2.drop(columns=["cast"])

credits2["crew"] = credits2["crew"].apply(ast.literal_eval)

In [14]:
# extraigo los nombres de los directores de cada pelicula  y los pongo en una lista. Luego cambio los valores de la columna crew por esas listas

def obtener_nombres_directores(lista):
    nombres_directores = []
    if isinstance(lista, list):
        for diccionario in lista:
            if isinstance(diccionario, dict) and diccionario.get("job") == "Director":
                nombre_director = diccionario.get("name")
                if nombre_director:
                    nombres_directores.append(nombre_director)
    return nombres_directores

In [15]:
credits2["crew"] = credits2["crew"].apply(obtener_nombres_directores)

In [16]:
# hacemos un merge para combinar las dataframe

movies_dataset_merged = movies_dataset.merge(credits2, on="id")

In [17]:
movies_dataset_merged.head()

,belongs_to_collection,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,release_year,return,crew
0,Toy Story Collection,30000000,"[Animation, Comedy, Family]",862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,[Pixar Animation Studios],[United States of America],1995-10-30,...,81.0,[English],Released,NaN,Toy Story,7.7,5415.0,1995,12.451801,[John Lasseter]
1,None,65000000,"[Adventure, Fantasy, Family]",8844,en,When siblings Judy and Peter discover an encha...,17.015539,"[TriStar Pictures, Teitler Film, Interscope Co...",[United States of America],1995-12-15,...,104.0,"[English, Français]",Released,Roll the dice and unleash the excitement!,Jumanji,6.9,2413.0,1995,4.043035,[Joe Johnston]
2,Grumpy Old Men Collection,0,"[Romance, Comedy]",15602,en,A family wedding reignites the ancient feud be...,11.7129,"[Warner Bros., Lancaster Gate]",[United States of America],1995-12-22,...,101.0,[English],Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,6.5,92.0,1995,0.000000,[Howard Deutch]
3,None,16000000,"[Comedy, Drama, Romance]",31357,en,"Cheated on, mistreated and stepped on, the wom...",3.859495,[Twentieth Century Fox Film Corporation],[United States of America],1995-12-22,...,127.0,[English],Released,Friends are the people who let you be yourself...,Waiting to Exhale,6.1,34.0,1995,5.090760,[Forest Whitaker]
4,Father of the Bride Collection,0,[Comedy],11862,en,Just when George Banks has recovered from his ...,8.387519,"[Sandollar Productions, Touchstone Pictures]",[United States of America],1995-02-10,...,106.0,[English],Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,5.7,173.0,1995,0.000000,[Charles Shyer]


In [18]:
movies_dataset_merged.to_csv('Dataset/df.csv', index=False)
